# Adversarial Attacks Function 
## from https://github.com/QData/TextAttack

In [7]:
#!pip install tensorflow_text
#!textattack list attack-recipes
#!pip install textattack
from textattack.augmentation import EmbeddingAugmenter,EasyDataAugmenter,WordNetAugmenter,CharSwapAugmenter,CheckListAugmenter,CLAREAugmenter
import time
import pandas as pd

In [8]:
t = "I do not understand transformers"
augmenter = CharSwapAugmenter()
augmenter.augment(t)


['I do not understand transformesr']

In [11]:
augmenter = EasyDataAugmenter(pct_words_to_swap=0.2, transformations_per_example=2)
s = 'What I do not understand, I can not create.'
augmenter.augment(s)

['What I not do understand, I can not create.',
 'What I do not understand, I non can not create.']

In [12]:

#     - WordNet synonym replacement
#         - Randomly replace words with their synonyms.
#     - Word deletion
#         - Randomly remove words from the sentence.
#     - Word order swaps
#         - Randomly swap the position of words in the sentence.
#     - Random synonym insertion
#         - Insert a random synonym of a random word at a random location.



def create_attacks(df,att_type,**kwargs):
    
    def easy_data(text):
        augmenter = EasyDataAugmenter(kwargs['pct_swap'], kwargs['num_trans'])
        return augmenter.augment(text)[0]
    
    def syno(text):
        augmenter = WordNetAugmenter()
        return augmenter.augment(text)[0]
    
    if att_type == "easy_data":
        df.text = df.text.apply(easy_data)
        
    elif att_type == "replace_synonym":
        df.text = df.text.apply(syno)

In [134]:
s = time.time()
create_attacks(df_combined.iloc[:5],att_type = "replace_synonym")
print(time.time()-s)

1.6966230869293213


# Prediction_String

In [193]:
def get_predStr(df):
    assert all(item in list(df) for item in ['label', 'text', 'doc_id']), "Please use a dataframe with correct columns"
    prediction_strings = []
    start_id = 1
    prev_doc = df.iloc[0].doc_id
    for (label, text, doc_id) in df[['label', 'text', 'doc_id']].itertuples(index=False):
        if doc_id != prev_doc:
            prev_doc = doc_id
            start_id = 0
        text_split = text.split()
        end_id = start_id + len(text_split)
        prediction_strings.append(
            [num for num in range(start_id, end_id)]
        )
        start_id = end_id   
    df['predictionString'] = prediction_strings